### Read .dat file generated from box and confirm impure probabilities

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from banditpy.core import Bandit2Arm

from banditpy.io import csv2ArmIO, dat2ArmIO

# from dat_to_csv import dat2ArmIO

# basedir = Path("Z:/Bandit2ArmData")
basedir = Path("D:/Data/mab/BGdataset/")
compiledir = Path("D:/Data/mab/BGdataset")
animals = [
    # "BGF0",
    # "BGF1",
    # "BGF2",
    # "BGF3",
    # "BGF4",
    # "BGM0",
    # "BGM1",
    # "BGM2",
    # "BGM3",
    # "BGM4",
    "BGM5",
]


def dat2compiledcsv(animal_name: str):
    print(animal_name)
    task = dat2ArmIO(basedir / animal_name / "raw_data")
    # task = csv2ArmIO(basedir / animal_name / "raw_data")
    df = task.to_df()
    df.to_csv(compiledir / animal_name / (animal_name + ".csv"))
    print(f"Saved {animal_name} to compiled csv.")


[dat2compiledcsv(animal_name) for animal_name in animals]

BGM5


C:\Users\asheshlab\Documents\Codes\BanditPy\banditpy\io\datio.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(fp, sep=",", header=None) for fp in files]
C:\Users\asheshlab\Documents\Codes\BanditPy\banditpy\io\datio.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(fp, sep=",", header=None) for fp in files]
C:\Users\asheshlab\Documents\Codes\BanditPy\banditpy\io\datio.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(fp, sep=",", header=None) for fp in files]
C:\Users\asheshlab\Documents\Codes\BanditPy\banditpy\io\datio.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(fp, sep=",", header=None) for fp in files]


Saved BGM5 to compiled csv.


[None]

In [ ]:
animals[0] / (animals[0].name + ".csv")

In [ ]:
np.unique(tasks[0].rewards, return_counts=True)

In [ ]:
from dat_to_csv import convert_dat_directory


data = convert_dat_directory(
    animals[0] / "raw_data", output_dir=animals[0], force=True, trial=True
)

In [ ]:
from neuropy import plotting
from datetime import datetime, timezone

fig = plotting.Fig(5, 1)
bin_edges = np.linspace(0, 24, 49)  # 0,0.5,...,24
bin_centers = bin_edges[:-1] + 0.25

for i, task in enumerate(tasks):

    dttime = task.to_df()["datetime"].to_numpy()
    time = np.array([datetime.fromtimestamp(ts, tz=timezone.utc) for ts in dttime])
    dt_series = pd.to_datetime(time, errors="coerce")
    hours = (
        dt_series.hour.to_numpy()
        + dt_series.minute.to_numpy() / 60.0
        + dt_series.second.to_numpy() / 3600.0
    )
    counts, _ = np.histogram(hours, bins=bin_edges)
    ax = fig.subplot(fig.gs[i])
    ax.bar(
        bin_centers, counts, width=0.48, align="center", edgecolor="k", linewidth=0.3
    )
    # ax.plot(time, ".")

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime


def read_datfile(folder):
    dfs = [pd.read_csv(fp, sep=",", header=None) for fp in sorted(folder.glob("*.dat"))]
    data = pd.concat(dfs, ignore_index=True)
    return data


df = read_datfile(animals[0])

arr = df[5].to_numpy()
dt = [datetime.fromtimestamp(ts) for ts in arr]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(tasks[0].datetime)

In [ ]:
from datetime import datetime, timezone
import pandas as pd

val = 1756913711000
dt = datetime.fromtimestamp(val / 1000, tz=timezone.utc)

pd.to_datetime(val, unit="ms", origin="unix", utc=True)

In [ ]:
import matplotlib.pyplot as plt
from banditpy.plots import plot_trial_by_trial_2Arm
from neuropy import plotting
from scipy import stats

fig = plotting.Fig(5, 4)
animal_names = ["BGM0", "BGM1", "BGM2"]

for i, task in enumerate(tasks):

    probs = task.probs[task.is_session_start, :]
    probs_corr = stats.pearsonr(probs[:, 0].astype(float), probs[:, 1].astype(float))
    print(probs_corr[0])

    p1_bins = np.linspace(0, 0.9, 10) + 0.05
    p2_bins = np.linspace(0, 0.9, 10) + 0.05
    H, xedges, yedges, _ = stats.binned_statistic_2d(
        probs[:, 0].astype(float),
        probs[:, 1].astype(float),
        values=probs[:, 0],
        statistic="count",
        bins=[p1_bins, p2_bins],
    )

    choices = task.choices.copy()
    # choices[choices == 1] = 2
    # choices[task.choices == 2] = 1
    # task.choices = choices

    perf = task.filter_by_trials(
        min_trials=100, clip_max=100
    ).get_optimal_choice_probability()

    ax = fig.subplot(fig.gs[i, 0])
    ax.plot(perf, color="k")
    # task.plot_trial_by_trial(ax=ax)
    # ax.set_ylim(0.1, 0.7)
    ax.set_xlabel("Trial_id")
    ax.set_ylabel("Choice (High)")
    ax.set_title(f"{animal_names[i]}'s performance")

    ax2 = fig.subplot(fig.gs[i, 1])
    cplot = ax2.pcolormesh(
        xedges,
        yedges,
        H.T,
        cmap="hot",
        vmin=0,
        # vmax=14,
        shading="auto",
    )
    cb = plt.colorbar(cplot, ax=ax2, shrink=0.5)
    # ax2.imshow(prob_mat)
    ax2.set_xlabel("Prob1")
    ax2.set_ylabel("Prob2")
    ax2.set_title(f"{animal_names[i]}'s probability combinations")
    # ax2.set_xticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    # ax2.set_yticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    cb.set_label("Counts")

# plt.plot(perf)
# plot_trial_by_trial_2Arm(task)

In [ ]:
probs[:, 0]

In [ ]:
tasks[1].n_sessions

In [ ]:
tasks[1].ntrials_session

In [ ]:
tasks[1].to_df()

In [ ]:
np.diff(tasks[2].datetime[tasks[2].is_session_start])

In [ ]:
tasks[2].ntrials_session

### Debugging

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from banditpy.core import Bandit2Arm
from banditpy.io import dat2ArmIO, csv2ArmIO


basedir = Path("D:/Data/mab/BGdataset")
# animals = [basedir / "BGM0", basedir / "BGM1", basedir / "BGM2"]
animals = [basedir / "BGF0"]

# tasks: list[Bandit2Arm] = [csv2ArmIO(folder / "raw_data") for folder in animals]

folder = Path("D:/Data/mab/BGdataset/BGF0/raw_data")
files = sorted(folder.glob("*.csv"))
print(len(files))
dfs = [pd.read_csv(fp, sep=",") for fp in files]
print(f"nfiles={len(dfs)}")
data = pd.concat(dfs, ignore_index=True)

data = data[
    (data["eventCode"].astype(str).str.contains("200"))
    & (data["chosenPort"].isin([1, 2]))
]

# blockId = data["blockId"].to_numpy()
# block_starts = np.where(np.diff(blockId, prepend=-1) != 0, 1, 0)
# sessionId = np.cumsum(block_starts)

# print(data["chosenPort"].unique())
print(data.size)

# b2a = Bandit2Arm(
#     probs=data[["port1Prob", "port2Prob"]].to_numpy(),
#     choices=data["chosenPort"].to_numpy(),
#     rewards=data["rewarded"].to_numpy(),
#     block_ids=blockId,
#     session_ids=sessionId,
#     # window_ids=data["window_id"].to_numpy(),
#     # datetime=data["datetime"].to_numpy(),
# )

### Debugging 2

In [ ]:
import pandas as pd
from pathlib import Path
from banditpy.core import Bandit2Arm
from banditpy.io import dat2ArmIO, csv2ArmIO


basedir = Path("D:/Data/mab/BGdataset")
# animals = [basedir / "BGM0", basedir / "BGM1", basedir / "BGM2"]
animals = [basedir / "BGM5"]

# tasks: list[Bandit2Arm] = [csv2ArmIO(folder / "raw_data") for folder in animals]

folder = Path("D:/Data/mab/BGdataset/BGF0/raw_data")


# Concatenate all .dat files
files = sorted(folder.glob("*.dat"))
print(len(files))
dfs = [pd.read_csv(fp, sep=",", header=None) for fp in files]
data = pd.concat(dfs, ignore_index=True)
# print(data.head())
# print(data[1].dtype)

code = data[0].astype(str)
arg = data[1].to_numpy().astype(int)
prob = data[2].to_numpy().astype(int)
dttime = data[5].to_numpy()

# print(arg)

# data["datetime"] = dttime
# # Probability updates
# data["p1_update"] = np.where((code == "83") & (arg == 1), prob, np.nan)
# data["p2_update"] = np.where((code == "83") & (arg == 2), prob, np.nan)
# data["p1"] = data["p1_update"].ffill()
# data["p2"] = data["p2_update"].ffill()

# # Session id increments when port1 prob line appears
# data["session_id"] = np.cumsum((code == "83") & (arg == 1))
# # Ensure first session starts at 1 (only if any updates exist)
# if data["session_id"].gt(0).any():
#     data.loc[data["session_id"] > 0, "session_id"] += 0

# # Port poke updates
# data["port_update"] = np.where((code == "81") & np.isin(arg, [1, 2]), arg, np.nan)
# data["port"] = data["port_update"].ffill()

# # Reward outcome mapping
# reward_map = {"51": 1, "-51": 0, "-52": -1}
# is_outcome = np.isin(code, list(reward_map.keys()))
# outcomes = data.loc[is_outcome, ["port", "p1", "p2", "session_id", "datetime", 0]]

# print(outcomes.columns)

# # Drop rows before first probabilities (port/prob may be NaN)
# outcomes = outcomes.dropna(subset=["port", "p1", "p2", "session_id"])

# outcomes["port"] = outcomes["port"].astype(int)
# outcomes["reward"] = outcomes[0].map(reward_map)
# behav = outcomes.loc[
#     :, ["port", "reward", "p1", "p2", "session_id", "datetime"]
# ].reset_index(drop=True)
# behav[behav["reward"].isin([0, 1])]

In [ ]:
p1_idx = prob[np.where((code == "83") & (arg == 1))[0]]

In [ ]:
p1_idx

In [ ]:
data2 = data[data[0].astype(str).isin(["83"])]

In [ ]:
data2.shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(data2[0])